In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.classification as metrics
from torchvision import transforms
from torchvision.datasets import MNIST

import numpy as np
import plotly.express as px
import os
from tqdm import tqdm

# Klasifikácia obrázkov

Už sme spoznali čo je MLP, a ako ho použiť na klasifikáciu jednoduchých dát. Teraz sa pozrieme na to ako použiť MLP na klasifikáciu obrázkov. Použijeme na to dataset MNIST, ktorý obsahuje obrázky rukou písaných číslic. Každý obrázok je 28x28 pixelov, a každý pixel je reprezentovaný číslom od 0 do 255, ktoré určuje jas. MNIST sa používa ako jednoduchý prototypovací dataset pre testovanie nových modelov NN.

MNIST sa nachádza priamo v PyTorchi, takže ho môžeme načítať jednoducho pomocou `torchvision.datasets`.

In [21]:
download = False
if not any(os.scandir("../data")):
    download = True
mnist_trainset = MNIST(root="../data", train=True, download=download, transform=transforms.Compose([transforms.ToTensor()]))
mnist_testset = MNIST(root="../data", train=False, download=download, transform=transforms.Compose([transforms.ToTensor()]))

print(f'Number of training examples: {len(mnist_trainset)}')
print(f'Number of test examples: {len(mnist_testset)}')

print(f'Image size: {mnist_trainset[0][0].shape}')
plot = px.imshow(mnist_trainset[0][0][0:28,0:28][0], color_continuous_scale='gray')
plot.show()

Number of training examples: 60000
Number of test examples: 10000
Image size: torch.Size([1, 28, 28])


# Definícia našej siete

Vytvorme si trochu modulárnejšiu a flexibilnejšiu verziu našej siete, ktorá bude fungovať na klasifikáciu obrázkov. Vytvoríme si triedu `MNIST_Mlp`, ktorá bude dediť z `nn.Module`. V tejto triede budeme definovať všetky vrstvy, ktoré budeme potrebovať. Využijeme `nn.ModuleList`, ktorý nám poskytuje možnosť vytvoriť si zoznam vrstiev, aké chceme. Aktivačné funkcie medzi vrstvami ponecháme `torch.relu` a na výstupe použijeme `torch.softmax`.

In [22]:
class MNIST_Mlp(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes):
        super(MNIST_Mlp, self).__init__()
        self.layer_sizes = [input_size] + hidden_sizes + [output_size]
        self.layers = nn.ModuleList([nn.Linear(self.layer_sizes[i], self.layer_sizes[i + 1]) for i in range(len(self.layer_sizes) - 1)])

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = torch.softmax(self.layers[-1](x), dim=1)
        return x

net = MNIST_Mlp(28 * 28, 10, [128, 64, 32, 16])
print(net)

MNIST_Mlp(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): Linear(in_features=16, out_features=10, bias=True)
  )
)


# Mini-batch training

Tentokrát má náš dataset veľkosť 60000 trénovacích dát a teda nie je jednoduché a v niektorých prípadoch ani možné spracovať všetky dáta naraz. Preto sa používa mini-batch training, ktorý nám umožní načítať dáta v menších dávkach. V PyTorchi je to jednoduché, stačí zavolať `DataLoader` a poskytnúť mu dataset, ktorý chceme načítať a veľkosť dávky. Výsledkom je `DataLoader` objekt, ktorý nám umožní iterovať cez dávky dát.

`labels` z datasetu MNIST sú uložené v tvare `0-9`, čo by bola úloha regresie. Pre zjednodušenie úlohy pre sieť, musíme transformovať dáta na klasifikačné a teda na one-hot encoding. V PyTorchi je to jednoduché pomocou `torch.nn.functional.one_hot`.

In [23]:
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=64, shuffle=False)

for inputs, labels in train_loader:
    print(inputs.shape)
    print(labels.shape)
    print(F.one_hot(labels, num_classes=10).shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 10])


# Train network

Poďme skúsiť natrénovať našu sieť. Opäť si potrebujeme nastaviť kritérium / loss funkciu, optimalizátor, a určíme si počet epôch ako dlho bude bežať tréning. V každej epoche prejdeme cez celý dataset - každý krok s mini-batchom sa nazýva step. V každej epoche si vypočítame loss a metriky na trénovacích dátach.

In [38]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = MNIST_Mlp(input_size=28 * 28, output_size=10, hidden_sizes=[128, 64, 32, 16])

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001)

precision = metrics.Precision(task='multiclass', num_classes=10)
recall = metrics.Recall(task='multiclass', num_classes=10)
f1_score = metrics.F1Score(task='multiclass', num_classes=10)

# ----------------------------------------

net.to(device)
precision.to(device)
recall.to(device)
f1_score.to(device)

train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=64, shuffle=False)

loss_values = []
precision_values = []
recall_values = []
f1_values = []

for epoch in range(5):
    batch_count = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}, loss: {loss_values[-1] if len(loss_values) > 0 else 0}'):
        inputs, labels = batch
        inputs = inputs.view(inputs.shape[0], -1).to(device)
        # use torch to one_hot encoding
        labels = F.one_hot(labels, num_classes=10).type(torch.float32).to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        if batch_count % 100 == 99:
            loss_values.append(loss.mean().detach().cpu().numpy())
            precision_values.append(precision(outputs, labels).detach().cpu().numpy())
            recall_values.append(recall(outputs, labels).detach().cpu().numpy())
            f1_values.append(f1_score(outputs, labels).detach().cpu().numpy())
        batch_count += 1

figure = px.line({'AdamW': loss_values})
figure.show()


Epoch 5, loss: 1.490175724029541: 100%|██████████| 938/938 [00:19<00:00, 48.80it/s]


In [ ]:
figure = px.line({'Precision': precision_values, 'Recall': recall_values, 'F1': f1_values})
figure.show()